In [18]:
import os
import keras
from keras.api.models import load_model, Sequential
from keras.api.layers import Dense, Flatten, Dropout
from keras.api.optimizers import Adam
from keras.api.losses import SparseCategoricalCrossentropy
from keras import layers, activations, Model

In [3]:
# We can customize the batch size, image height, and image width
batch_size  = 32
img_height  = 224
img_width   = 224
channel     = 3

image_size              = (img_height, img_width)
image_size_with_channel = (img_height, img_width, channel)

# Constant value
CHECKPOINTS = "checkpoints"
LATEST_MODEL_NAME = "latest_model.keras"

In [4]:
# Save the current model
def save_model(model, model_name: str = ""):
    try:
        model.save(CHECKPOINTS + os.sep + model_name + LATEST_MODEL_NAME)
    except Exception as e:
        print(e)
        raise e
    
    print("Model has succesfully saved")
    
def load_model_function(model_name):
    try:
        model_file = CHECKPOINTS + os.sep + model_name + LATEST_MODEL_NAME
        model = load_model(model_file)

        return model
    except Exception as e:
        raise e


In [5]:
(train_ds, val_ds) = keras.preprocessing.image_dataset_from_directory(
    directory="datasets/preprocessed_CLAHE",
    validation_split=0.2,
    subset="both",
    seed=69420,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 2426 files belonging to 3 classes.
Using 1941 files for training.
Using 485 files for validation.


I0000 00:00:1727024449.406444    6614 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727024449.802383    6614 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727024449.802426    6614 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727024449.813955    6614 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727024449.814198    6614 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [6]:
image_augmentation = {
    "training": Sequential([
        layers.RandomRotation(factor=0.2),
        layers.RandomFlip()
    ])
}

In [8]:
densenet_model = keras.applications.DenseNet169(
    include_top=False,
    classes=3,
    input_shape=(img_height, img_width, channel)
)

In [9]:
densenet_model.trainable = True

for layer in densenet_model.layers[:-4]:
    layer.trainable = False

In [21]:
inputs = keras.Input(shape=image_size_with_channel)
x = image_augmentation["training"](inputs)
x = keras.applications.densenet.preprocess_input(x)
x = densenet_model(x)
x = Flatten()(x)
x = Dense(256)(x)
x = Dropout(0.5)(x)
outputs = Dense(units=3, activation=activations.softmax)(x)
model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide_8 (TrueDivide)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ add_4 (Add)                     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide_9 (TrueDivide)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ densenet169 (Functional)        │ (None, 7, 7, 1664)     │    12,642,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 81536)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │    20,873,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,517,123 (127.86 MB)

 Trainable params: 20,914,435 (79.78 MB)

 Non-trainable params: 12,602,688 (48.08 MB)

In [16]:
def get_early_stopping():
    return keras.callbacks.EarlyStopping(patience=5)

In [22]:
model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

In [23]:
history = model.fit(train_ds, validation_data=val_ds, batch_size=batch_size, epochs=30, callbacks=[get_early_stopping()])

Epoch 1/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 44s 448ms/step - accuracy: 0.6126 - loss: 168.5963 - val_accuracy: 0.6103 - val_loss: 25.9332
Epoch 2/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 22s 368ms/step - accuracy: 0.6813 - loss: 39.9609 - val_accuracy: 0.7505 - val_loss: 12.6306
Epoch 3/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 23s 370ms/step - accuracy: 0.7125 - loss: 19.2123 - val_accuracy: 0.7464 - val_loss: 13.0768
Epoch 4/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 23s 371ms/step - accuracy: 0.7071 - loss: 18.8959 - val_accuracy: 0.7278 - val_loss: 13.1740
Epoch 5/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 23s 372ms/step - accuracy: 0.7435 - loss: 14.9663 - val_accuracy: 0.7588 - val_loss: 21.3589
Epoch 6/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 23s 373ms/step - accuracy: 0.7503 - loss: 14.8529 - val_accuracy: 0.7134 - val_loss: 20.6135
Epoch 7/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 23s 373ms/step - accuracy: 0.7484 - loss: 10.8539 - val_accuracy: 0.7856 - val_loss: 8.3841
Epoch 8/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 23s 376ms/step - accuracy: 0.7582 - loss: 10.